In [86]:
import numpy as np
import os
from datasets import load_dataset
import subprocess
from pandas import DataFrame
from pathlib import Path
import json

In [187]:
ds = load_dataset("dz1/CodeScore-MBPP-ET")
CodeScore_df = DataFrame(ds['train'][89:99])
CodeScore_df
# ds = load_dataset("evalplus/humanevalplus")
# human_eval_df = DataFrame(ds['test'][89:90])

,text,code,task_id,test_setup_code,test_list,challenge_test_list,entry_point
0,Write a python function to find the length of ...,def len_log(list1):\r\n max=len(list1[0])\r...,90,,"[assert len_log([""python"",""PHP"",""bigdata""]) ==...",[],len_log
1,Write a function to check if a substring is pr...,"def find_substring(str1, sub_str):\r\n if an...",91,,"[assert find_substring([""red"", ""black"", ""white...",[],find_substring
2,Write a function to check whether the given nu...,def is_undulating(n): \r\n\tif (len(n) <= 2): ...,92,,"[assert is_undulating(""1212121"") == True, asse...",[],is_undulating
3,Write a function to calculate the value of 'a'...,"def power(a,b):\r\n\tif b==0:\r\n\t\treturn 1\...",93,,"[assert power(3,4) == 81, assert power(2,3) ==...",[],power
4,Write a function to extract the index minimum ...,from operator import itemgetter \r\ndef index_...,94,,"[assert index_minimum([('Rash', 143), ('Manjee...",[],index_minimum
5,Write a python function to find the minimum le...,def Find_Min_Length(lst): \r\n minLength =...,95,,"[assert Find_Min_Length([[1],[1,2]]) == 1, ass...",[],Find_Min_Length
6,Write a python function to find the number of ...,def divisor(n):\r\n for i in range(n):\r\n ...,96,,"[assert divisor(15) == 4 , assert divisor(12) ...",[],divisor
7,Write a function to find frequency count of li...,def frequency_lists(list1):\r\n list1 = [it...,97,,"[assert frequency_lists([[1, 2, 3, 2], [4, 5, ...",[],frequency_lists
8,Write a function to multiply all the numbers i...,def multiply_num(numbers): \r\n total = 1\...,98,,"[assert multiply_num((8, 2, 3, -1, 7))==-67.2,...",[],multiply_num
9,Write a function to convert the given decimal ...,def decimal_to_binary(n): \r\n return bin(n...,99,,"[assert decimal_to_binary(8) == '1000', assert...",[],decimal_to_binary


In [166]:
selfrepair_prompt = """
You are an expert Python programmer.
Solve the following problem.

Problem:
{problem_statement}

The name of the entry_point function should be "{entry_point}"

If there are mistakes, repair them.

Output solution in this exact JSON format:

{{
  "initial_code": "Write valid Python code implementing the plan.",
  "repair_text": "Describe any mistakes in the initial_code, and explain how you'll repair them in the final code"
  "code": "Write the final valid Python code implementing the plan.",
}}

Important:
- The 'code' must be valid Python.
- Do NOT include markdown fences (no ```python or ```).
- Keep the JSON strictly valid.
"""

scot_prompt = """
You are an expert Python programmer.
Solve the following problem, reasoning step-by-step in a structured way.

Problem:
{problem_statement}

The name of the entry_point function should be "{entry_point}"

Output your reasoning and solution in this exact JSON format:

{{
  "understanding": "Explain the problem in your own words.",
  "plan": "Describe how you'll solve it.",
  "reasoning_steps": ["Step 1...", "Step 2...", "..."],
  "code": "Write valid Python code implementing the plan.",
}}

Important:
- The 'code' must be valid Python.
- Do NOT include markdown fences (no ```python or ```).
- Keep the JSON strictly valid.
"""
CodeScore_df['text_selfrepair'] = CodeScore_df.apply(lambda row: selfrepair_prompt.format(problem_statement=row.text, entry_point=row.entry_point), axis = 1)
CodeScore_df['text_scot'] = CodeScore_df.apply(lambda row: scot_prompt.format(problem_statement=row.text,entry_point=row.entry_point), axis = 1)

for test_idx in range(10): 
    print(CodeScore_df['text_selfrepair'][test_idx])


You are an expert Python programmer.
Solve the following problem.

Problem:
Write a python function to find the length of the longest word.

The name of the entry_point function should be "len_log"

If there are mistakes, repair them.

Output solution in this exact JSON format:

{
  "initial_code": "Write valid Python code implementing the plan.",
  "repair_text": "Describe any mistakes in the initial_code, and explain how you'll repair them in the final code"
  "code": "Write the final valid Python code implementing the plan.",
}

Important:
- The 'code' must be valid Python.
- Do NOT include markdown fences (no ```python or ```).
- Keep the JSON strictly valid.


You are an expert Python programmer.
Solve the following problem.

Problem:
Write a function to check if a substring is present in a given list of string values.

The name of the entry_point function should be "find_substring"

If there are mistakes, repair them.

Output solution in this exact JSON format:

{
  "initial_cod

In [159]:
from pathlib import Path
import textwrap

# Create Folders
for folder_name in ['chatgpt_written_code/scot','chatgpt_written_code/selfrepair','gemini_written_code/scot','gemini_written_code/selfrepair']:
    for problem_name in CodeScore_df['entry_point']:
        folder_path = Path(folder_name)
        file_name = problem_name + ".py"
        file_path = folder_path / file_name

        # Dedent the content to remove extra leading spaces
        file_content = textwrap.dedent("""\
            # This is a sample Python file created with pathlib.

            def hello_world():
                print("Hello, World!")

            if __name__ == "__main__":
                hello_world()
        """)

        try:
            folder_path.mkdir(parents=True, exist_ok=True)
            file_path.write_text(file_content)
            print(f"Successfully created '{file_path}'")
        except Exception as e:
            print(f"An error occurred: {e}")


Successfully created 'chatgpt_written_code\scot\len_log.py'
Successfully created 'chatgpt_written_code\scot\find_substring.py'
Successfully created 'chatgpt_written_code\scot\is_undulating.py'
Successfully created 'chatgpt_written_code\scot\power.py'
Successfully created 'chatgpt_written_code\scot\index_minimum.py'
Successfully created 'chatgpt_written_code\scot\Find_Min_Length.py'
Successfully created 'chatgpt_written_code\scot\divisor.py'
Successfully created 'chatgpt_written_code\scot\frequency_lists.py'
Successfully created 'chatgpt_written_code\scot\multiply_num.py'
Successfully created 'chatgpt_written_code\scot\decimal_to_binary.py'
Successfully created 'chatgpt_written_code\scot\next_smallest_palindrome.py'
Successfully created 'chatgpt_written_code\selfrepair\len_log.py'
Successfully created 'chatgpt_written_code\selfrepair\find_substring.py'
Successfully created 'chatgpt_written_code\selfrepair\is_undulating.py'
Successfully created 'chatgpt_written_code\selfrepair\power.py'


In [ ]:
import google.generativeai as genai

def gemini_generate(prompt_technique,problemrange):
    genai.configure(api_key=GOOGLE_API_KEY)
    model = genai.GenerativeModel("gemini-2.5-flash")

    for test_idx in range(problemrange):
        response = model.generate_content(CodeScore_df["text_"+prompt_technique][test_idx])
        print(response.text)

        # Step 1: Remove Markdown fences if present
        raw_response = response.text
        cleaned = raw_response.strip()
        if cleaned.startswith("```"):
            cleaned = "\n".join(cleaned.split("\n")[1:-1])  # remove first and last lines

        # Step 2: Parse JSON
        data = json.loads(cleaned)

        # Step 3: Extract the 'code' field
        code_text = data["code"]

        # Step 4: Write to file
        folder_path = Path("gemini_written_code/"+prompt_technique)
        folder_path.mkdir(parents=True, exist_ok=True)

        file_path = folder_path / (CodeScore_df['entry_point'][test_idx] + ".py")
        file_path.write_text(code_text, encoding="utf-8")

        print(f"Successfully wrote code to '{file_path}'")

In [206]:
def run_tests_visible(model_name,prompt_technique,problem_idx):
    entry_point = CodeScore_df['entry_point'][problem_idx]
    problem_test_list = CodeScore_df['test_list'][problem_idx]

    # Read the Python code from the file
    with open(model_name+"_written_code/"+prompt_technique+"/"+entry_point+".py", "r", encoding="utf-8") as f:
        problem_code = f.read()

    # Prepare namespace and execute the code
    namespace = {}
    exec(problem_code, namespace)

    for test in problem_test_list:
        try:
            exec(test, namespace)
            print(f"✅ Passed: {test}")
        except Exception as e:
            print(f"❌ Failed: {test}")
            print("   Error:", e)

# TEST EVERY TEST

def get_tests_passed(model_name,prompt_technique):
    folder_name= model_name+"_written_code/"+prompt_technique
    folder_path = Path(folder_name)
    problems_passed = []
    
    for idx, problem_name in enumerate(CodeScore_df['entry_point']):
        # Build the file path
        file_name = f"{problem_name}.py"
        file_path = folder_path / file_name

        if not file_path.exists():
            print(f"File not found: {file_path}")
            continue

        # Read the Python code from the file
        with open(file_path, "r", encoding="utf-8") as f:
            problem_code = f.read()

        # Get the corresponding tests from the dataframe
        problem_test_list = CodeScore_df['test_list'][idx]

        # Prepare namespace and execute the code
        namespace = {}
        try:
            exec(problem_code, namespace)
        except Exception as e:
            print(f"Error executing code for {problem_name} in {folder_name}: {e}")
            continue

        # Run tests
        passed = 0
        total = len(problem_test_list)

        for test in problem_test_list:
            try:
                exec(test, namespace)
                passed += 1
            except Exception:
                pass  # test failed
        
        if passed >= total: #passed all tests
            problems_passed.append(problem_name)

        print(f"{folder_name}/{file_name}: Passed {passed}/{total} tests")

    return problems_passed


In [162]:
gemini_generate("scot",10)

{
  "understanding": "The problem asks us to write a Python function named `len_log` that takes an input and determines the length of the longest word within it. Based on typical word-related problems, the most common input format for this scenario is a string representing a sentence or a phrase. The function should correctly identify individual words, calculate their lengths, and return the maximum length found. Edge cases like empty input strings or strings containing only whitespace should be handled gracefully, returning 0 in such situations.",
  "plan": "1. Define the function `len_log` that accepts a single string argument, let's call it `text`.\n2. Utilize the `str.split()` method on the input `text`. When called without arguments, `text.split()` efficiently handles various whitespace scenarios (single spaces, multiple spaces, leading/trailing spaces) and returns a list of non-empty strings, which are our words. If the input `text` is empty or contains only whitespace, `text.spl

In [169]:
gemini_generate("selfrepair",10)

{
  "initial_code": "def len_log(sentence):\n    words = sentence.split() # Splits the sentence into words using whitespace as delimiter.\n    # This will raise a ValueError if 'words' is an empty list (e.g., for an empty sentence).\n    return max(len(word) for word in words)",
  "repair_text": "The initial code uses a concise approach with `max(len(word) for word in words)`. While elegant, this code has a critical flaw for edge cases: if the input `sentence` is an empty string, or contains only whitespace characters (e.g., \"   \"), `sentence.split()` will return an empty list `[]`. Calling `max()` on an empty iterable (like an empty generator expression) raises a `ValueError`.\n\nThe repair involves adding a check to see if the `words` list is empty before attempting to call `max()`. If `words` is empty, it means there are no words in the sentence, and therefore the length of the longest word should be 0. This makes the function robust against all valid inputs.",
  "code": "def len_

# Manual Generation of CHATGPT CODE:

In [200]:
for i in range(10):
    print(CodeScore_df['text'][i])

Write a python function to find the length of the longest word.
Write a function to check if a substring is present in a given list of string values.
Write a function to check whether the given number is undulating or not.
Write a function to calculate the value of 'a' to the power 'b'.
Write a function to extract the index minimum value record from the given tuples.
Write a python function to find the minimum length of sublist.
Write a python function to find the number of divisors of a given integer.
Write a function to find frequency count of list of lists.
Write a function to multiply all the numbers in a list and divide with the length of the list.
Write a function to convert the given decimal number to its binary equivalent.


In [ ]:
# For Transferring Manually to the Claude Model
for i in range(10):
    print(CodeScore_df['text_scot'][i])
    print("_____________")


You are an expert Python programmer.
Solve the following problem, reasoning step-by-step in a structured way.

Problem:
Write a python function to find the length of the longest word.

The name of the entry_point function should be "len_log"

Output your reasoning and solution in this exact JSON format:

{
  "understanding": "Explain the problem in your own words.",
  "plan": "Describe how you'll solve it.",
  "reasoning_steps": ["Step 1...", "Step 2...", "..."],
  "code": "Write valid Python code implementing the plan.",
}

Important:
- The 'code' must be valid Python.
- Do NOT include markdown fences (no ```python or ```).
- Keep the JSON strictly valid.

_____________

You are an expert Python programmer.
Solve the following problem, reasoning step-by-step in a structured way.

Problem:
Write a function to check if a substring is present in a given list of string values.

The name of the entry_point function should be "find_substring"

Output your reasoning and solution in this exa

In [192]:
print("def len_log(words):\n    if isinstance(words, str):\n        words = words.split()\n    if not words:\n        return 0\n    return max(len(word) for word in words)")
print("def find_substring(string_list, substring):\n    for string in string_list:\n        if substring in string:\n            return True\n    return False")
print("def is_undulating(n):\n    s = str(n)\n    if len(s) < 3:\n        return False\n    \n    if len(set(s)) != 2:\n        return False\n    \n    for i in range(len(s) - 1):\n        if s[i] == s[i + 1]:\n            return False\n    \n    return True")
print("def power(a, b):\n    return a ** b")
print("def index_minimum(tuples_list):\n    if not tuples_list:\n        return None\n    return tuple(min(values) for values in zip(*tuples_list))")
print("def Find_Min_Length(lst):\n    if not lst:\n        return 0\n    return min(len(sublist) for sublist in lst)")
print("def divisor(n):\n    if n <= 0:\n        return 0\n    count = 0\n    i = 1\n    while i * i <= n:\n        if n % i == 0:\n            if i * i == n:\n                count += 1\n            else:\n                count += 2\n        i += 1\n    return count")
print("def frequency_lists(list_of_lists):\n    from collections import Counter\n    freq = Counter(tuple(sublist) for sublist in list_of_lists)\n    return dict(freq)")
print("def multiply_num(numbers):\n    if not numbers:\n        return 0\n    product = 1\n    for num in numbers:\n        product *= num\n    return product / len(numbers)")
print("def decimal_to_binary(n):\n    if n >= 0:\n        return bin(n)[2:]\n    else:\n        return '-' + bin(n)[3:]")

def len_log(words):
    if isinstance(words, str):
        words = words.split()
    if not words:
        return 0
    return max(len(word) for word in words)
def find_substring(string_list, substring):
    for string in string_list:
        if substring in string:
            return True
    return False
def is_undulating(n):
    s = str(n)
    if len(s) < 3:
        return False
    
    if len(set(s)) != 2:
        return False
    
    for i in range(len(s) - 1):
        if s[i] == s[i + 1]:
            return False
    
    return True
def power(a, b):
    return a ** b
def index_minimum(tuples_list):
    if not tuples_list:
        return None
    return tuple(min(values) for values in zip(*tuples_list))
def Find_Min_Length(lst):
    if not lst:
        return 0
    return min(len(sublist) for sublist in lst)
def divisor(n):
    if n <= 0:
        return 0
    count = 0
    i = 1
    while i * i <= n:
        if n % i == 0:
            if i * i == n:
                count += 1
 

# TESTS

In [ ]:
get_tests_passed("claude","selfrepair")

claude_run_1_written_code/selfrepair/len_log.py: Passed 102/102 tests
claude_run_1_written_code/selfrepair/find_substring.py: Passed 102/102 tests
claude_run_1_written_code/selfrepair/is_undulating.py: Passed 102/102 tests
claude_run_1_written_code/selfrepair/power.py: Passed 102/102 tests
claude_run_1_written_code/selfrepair/index_minimum.py: Passed 0/102 tests
claude_run_1_written_code/selfrepair/Find_Min_Length.py: Passed 102/102 tests
claude_run_1_written_code/selfrepair/divisor.py: Passed 102/102 tests
claude_run_1_written_code/selfrepair/frequency_lists.py: Passed 0/102 tests
claude_run_1_written_code/selfrepair/multiply_num.py: Passed 102/102 tests
claude_run_1_written_code/selfrepair/decimal_to_binary.py: Passed 3/102 tests


['len_log',
 'find_substring',
 'is_undulating',
 'power',
 'Find_Min_Length',
 'divisor',
 'multiply_num']

In [ ]:
get_tests_passed("claude","scot")

claude_written_code/scot/len_log.py: Passed 102/102 tests
claude_written_code/scot/find_substring.py: Passed 0/102 tests
claude_written_code/scot/is_undulating.py: Passed 102/102 tests
claude_written_code/scot/power.py: Passed 102/102 tests
claude_written_code/scot/index_minimum.py: Passed 0/102 tests
claude_written_code/scot/Find_Min_Length.py: Passed 102/102 tests
claude_written_code/scot/divisor.py: Passed 102/102 tests
claude_written_code/scot/frequency_lists.py: Passed 0/102 tests
claude_written_code/scot/multiply_num.py: Passed 102/102 tests
claude_written_code/scot/decimal_to_binary.py: Passed 3/102 tests


['len_log',
 'is_undulating',
 'power',
 'Find_Min_Length',
 'divisor',
 'multiply_num']

In [ ]:
get_tests_passed("gemini","scot")

gemini_written_code/scot/len_log.py: Passed 0/102 tests
gemini_written_code/scot/find_substring.py: Passed 102/102 tests
gemini_written_code/scot/is_undulating.py: Passed 85/102 tests
gemini_written_code/scot/power.py: Passed 102/102 tests
gemini_written_code/scot/index_minimum.py: Passed 0/102 tests
gemini_written_code/scot/Find_Min_Length.py: Passed 102/102 tests
gemini_written_code/scot/divisor.py: Passed 102/102 tests
gemini_written_code/scot/frequency_lists.py: Passed 0/102 tests
gemini_written_code/scot/multiply_num.py: Passed 102/102 tests
gemini_written_code/scot/decimal_to_binary.py: Passed 3/102 tests


['find_substring', 'power', 'Find_Min_Length', 'divisor', 'multiply_num']

In [196]:
get_tests_passed("gemini","selfrepair")

gemini_written_code/selfrepair/len_log.py: Passed 0/102 tests
gemini_written_code/selfrepair/find_substring.py: Passed 0/102 tests
gemini_written_code/selfrepair/is_undulating.py: Passed 102/102 tests
gemini_written_code/selfrepair/power.py: Passed 102/102 tests
gemini_written_code/selfrepair/index_minimum.py: Passed 0/102 tests
gemini_written_code/selfrepair/Find_Min_Length.py: Passed 102/102 tests
gemini_written_code/selfrepair/divisor.py: Passed 102/102 tests
gemini_written_code/selfrepair/frequency_lists.py: Passed 0/102 tests
gemini_written_code/selfrepair/multiply_num.py: Passed 102/102 tests
gemini_written_code/selfrepair/decimal_to_binary.py: Passed 3/102 tests


['is_undulating', 'power', 'Find_Min_Length', 'divisor', 'multiply_num']

In [ ]:
selfrepair_prompt_altered = """
You are an expert Python programmer.
Solve the following problem.

Problem:
{problem_statement}

The name of the entry_point function should be "{entry_point}"

If there are mistakes, repair them.

Output solution in this exact JSON format:

{{
  "initial_code": "Write valid Python code implementing the plan.",
  "repair_text": "Describe any mistakes in the initial_code, and explain how you'll repair them in the final code"
  "code": "Write the final valid Python code implementing the plan.",
}}

Important:
- The 'code' must be valid Python.
- Do NOT include markdown fences (no ```python or ```).
- Keep the JSON strictly valid.
"""

scot_prompt = """
You are an expert Python programmer.
Solve the following problem, reasoning step-by-step in a structured way.

Problem:
{problem_statement}

The name of the entry_point function should be "{entry_point}"

Output your reasoning and solution in this exact JSON format:

{{
  "understanding": "Explain the problem in your own words.",
  "plan": "Describe how you'll solve it.",
  "reasoning_steps": ["Step 1...", "Step 2...", "..."],
  "code": "Write valid Python code implementing the plan.",
}}

Important:
- The 'code' must be valid Python.
- Do NOT include markdown fences (no ```python or ```).
- Keep the JSON strictly valid.
"""
CodeScore_df['text_selfrepair'] = CodeScore_df.apply(lambda row: selfrepair_prompt.format(problem_statement=row.text, entry_point=row.entry_point), axis = 1)
CodeScore_df['text_scot'] = CodeScore_df.apply(lambda row: scot_prompt.format(problem_statement=row.text,entry_point=row.entry_point), axis = 1)


In [213]:
run_tests_visible("gemini_run_1","selfrepair",7)

❌ Failed: assert frequency_lists([[1, 2, 3, 2], [4, 5, 6, 2], [7, 8, 9, 5]])=={1: 1, 2: 3, 3: 1, 4: 1, 5: 2, 6: 1, 7: 1, 8: 1, 9: 1}
   Error: 
❌ Failed: assert frequency_lists([[1,2,3,4],[5,6,7,8],[9,10,11,12]])=={1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1,10:1,11:1,12:1}
   Error: 
❌ Failed: assert frequency_lists([[20,30,40,17],[18,16,14,13],[10,20,30,40]])=={20:2,30:2,40:2,17: 1,18:1, 16: 1,14: 1,13: 1, 10: 1}
   Error: 
❌ Failed: assert frequency_lists([[3, 7, 1, 5], [2, 6, 2, 6], [12, 4, 8, 8]]) == {3: 1, 7: 1, 1: 1, 5: 1, 2: 2, 6: 2, 12: 1, 4: 1, 8: 2}
   Error: 
❌ Failed: assert frequency_lists([[2, 1, 2, 4], [3, 5, 7, 4], [3, 12, 9, 2]]) == {2: 3, 1: 1, 4: 2, 3: 2, 5: 1, 7: 1, 12: 1, 9: 1}
   Error: 
❌ Failed: assert frequency_lists([[6, 3, 2, 3], [8, 2, 10, 3], [5, 12, 5, 5]]) == {6: 1, 3: 3, 2: 2, 8: 1, 10: 1, 5: 3, 12: 1}
   Error: 
❌ Failed: assert frequency_lists([[4, 4, 5, 5], [7, 10, 7, 5], [12, 4, 13, 9]]) == {4: 3, 5: 3, 7: 2, 10: 1, 12: 1, 13: 1, 9: 1}
   E

In [ ]:
run_tests_visible("gemini_run_1","scot",2)


✅ Passed: assert is_undulating("1212121") == True
✅ Passed: assert is_undulating("1991") == False
✅ Passed: assert is_undulating("121") == True
✅ Passed: assert is_undulating("821762331") == False
✅ Passed: assert is_undulating("320097251") == False
✅ Passed: assert is_undulating("55719077819") == False
✅ Passed: assert is_undulating("4478552") == False
✅ Passed: assert is_undulating("50082161454") == False
✅ Passed: assert is_undulating("76309042") == False
✅ Passed: assert is_undulating("314190063970") == False
✅ Passed: assert is_undulating("053203291") == False
✅ Passed: assert is_undulating("888587793") == False
✅ Passed: assert is_undulating("59445720") == False
❌ Failed: assert is_undulating("052") == False
   Error: 
❌ Failed: assert is_undulating("4847") == False
   Error: 
✅ Passed: assert is_undulating("418122655672") == False
✅ Passed: assert is_undulating("098160505632") == False
✅ Passed: assert is_undulating("087763313") == False
✅ Passed: assert is_undulating("693859374

## SOlutions

In [ ]:
scot_prompt_ambiguity = """
You are an expert Python programmer.
Solve the following problem, reasoning step-by-step in a structured way.

Problem:
{problem_statement}

Undulating refers to alternating two numbers, not multiple. 

Here is a single test to understand the format of the function: "{test_list_sample}"

The name of the entry_point function should be "{entry_point}"

Output your reasoning and solution in this exact JSON format:

{{
  "understanding": "Explain the problem in your own words.",
  "plan": "Describe how you'll solve it.",
  "reasoning_steps": ["Step 1...", "Step 2...", "..."],
  "code": "Write valid Python code implementing the plan.",
}}

Important:
- The 'code' must be valid Python.
- Do NOT include markdown fences (no ```python or ```).
- Keep the JSON strictly valid.
"""

CodeScore_df['text_scot_ambiguity'] = CodeScore_df.apply(lambda row: scot_prompt_ambiguity.format(problem_statement=row.text, entry_point=row.entry_point,test_list_sample=row.test_list[0]), axis = 1)

In [241]:
gemini_generate("scot_ambiguity",3)

{
  "understanding": "The problem requires writing a Python function called `len_log` that takes a list of words (strings) as input. The function should iterate through this list and determine the length of the longest word present. The final output should be an integer representing this maximum length. The statement about 'undulating' seems irrelevant to this specific problem and will be ignored.",
  "plan": "1. Initialize a variable, let's call it `max_length`, to 0. This variable will keep track of the longest word length found so far. 2. Iterate through each word in the input list. 3. For each word, calculate its length using the built-in `len()` function. 4. Compare the current word's length with `max_length`. If the current word's length is greater than `max_length`, update `max_length` with this new, longer length. 5. After iterating through all words in the list, return the final value of `max_length`.",
  "reasoning_steps": [
    "Define the function `len_log` that takes one p

In [242]:
run_tests_visible("gemini","scot_ambiguity",2)

✅ Passed: assert is_undulating("1212121") == True
✅ Passed: assert is_undulating("1991") == False
✅ Passed: assert is_undulating("121") == True
✅ Passed: assert is_undulating("821762331") == False
✅ Passed: assert is_undulating("320097251") == False
✅ Passed: assert is_undulating("55719077819") == False
✅ Passed: assert is_undulating("4478552") == False
✅ Passed: assert is_undulating("50082161454") == False
✅ Passed: assert is_undulating("76309042") == False
✅ Passed: assert is_undulating("314190063970") == False
✅ Passed: assert is_undulating("053203291") == False
✅ Passed: assert is_undulating("888587793") == False
✅ Passed: assert is_undulating("59445720") == False
✅ Passed: assert is_undulating("052") == False
✅ Passed: assert is_undulating("4847") == False
✅ Passed: assert is_undulating("418122655672") == False
✅ Passed: assert is_undulating("098160505632") == False
✅ Passed: assert is_undulating("087763313") == False
✅ Passed: assert is_undulating("6938593744") == False
✅ Passed:

In [243]:
selfrepair_prompt_with_test = """
You are an expert Python programmer.
Solve the following problem.

Problem:
{problem_statement}

The name of the entry_point function should be "{entry_point}"

Here is a single test to understand the format of the function: "{test_list_sample}"

If there are mistakes, repair them.

Output solution in this exact JSON format:

{{
  "initial_code": "Write valid Python code implementing the plan.",
  "repair_text": "Describe any mistakes in the initial_code, and explain how you'll repair them in the final code"
  "code": "Write the final valid Python code implementing the plan.",
}}

Important:
- The 'code' must be valid Python.
- Do NOT include markdown fences (no ```python or ```).
- Keep the JSON strictly valid.
"""
scot_prompt_with_test = """
You are an expert Python programmer.
Solve the following problem, reasoning step-by-step in a structured way.

Problem:
{problem_statement}

The name of the entry_point function should be "{entry_point}"

Here is a single test to understand the format of the function: "{test_list_sample}"

Output your reasoning and solution in this exact JSON format:

{{
  "understanding": "Explain the problem in your own words.",
  "plan": "Describe how you'll solve it.",
  "reasoning_steps": ["Step 1...", "Step 2...", "..."],
  "code": "Write valid Python code implementing the plan.",
}}

Important:
- The 'code' must be valid Python.
- Do NOT include markdown fences (no ```python or ```).
- Keep the JSON strictly valid.
"""

CodeScore_df['text_scot_with_test'] = CodeScore_df.apply(lambda row: scot_prompt_with_test.format(problem_statement=row.text, entry_point=row.entry_point,test_list_sample=row.test_list[0]), axis = 1)
CodeScore_df['text_selfrepair_with_test'] = CodeScore_df.apply(lambda row: selfrepair_prompt_with_test.format(problem_statement=row.text, entry_point=row.entry_point,test_list_sample=row.test_list[0]), axis = 1)


In [244]:
gemini_generate("scot_with_test",10)
gemini_generate("selfrepair_with_test",10)

{
  "understanding": "The problem asks us to write a Python function that takes a list of strings (words) as input. The function should iterate through this list and determine which word has the greatest length. Finally, it should return this maximum length as an integer. For example, if the input is ['apple', 'banana', 'kiwi'], the longest word is 'banana' with a length of 6, so the function should return 6.",
  "plan": "To find the length of the longest word, I will initialize a variable to keep track of the maximum length found so far. Then, I will iterate through each word in the provided list. For each word, I will calculate its length and compare it with the current maximum length. If the current word's length is greater, I will update the maximum length. After checking all words, the stored maximum length will be the answer.",
  "reasoning_steps": [
    "Define the function `len_log` which takes one argument, `words_list`, expected to be a list of strings.",
    "Handle the edge

In [246]:
get_tests_passed("gemini","scot_with_test")

gemini_written_code/scot_with_test/len_log.py: Passed 102/102 tests
gemini_written_code/scot_with_test/find_substring.py: Passed 102/102 tests
gemini_written_code/scot_with_test/is_undulating.py: Passed 102/102 tests
gemini_written_code/scot_with_test/power.py: Passed 102/102 tests
gemini_written_code/scot_with_test/index_minimum.py: Passed 102/102 tests
gemini_written_code/scot_with_test/Find_Min_Length.py: Passed 102/102 tests
gemini_written_code/scot_with_test/divisor.py: Passed 102/102 tests
gemini_written_code/scot_with_test/frequency_lists.py: Passed 102/102 tests
gemini_written_code/scot_with_test/multiply_num.py: Passed 102/102 tests
gemini_written_code/scot_with_test/decimal_to_binary.py: Passed 3/102 tests


['len_log',
 'find_substring',
 'is_undulating',
 'power',
 'index_minimum',
 'Find_Min_Length',
 'divisor',
 'frequency_lists',
 'multiply_num']

In [247]:
get_tests_passed("gemini","selfrepair_with_test")

gemini_written_code/selfrepair_with_test/len_log.py: Passed 102/102 tests
gemini_written_code/selfrepair_with_test/find_substring.py: Passed 102/102 tests
gemini_written_code/selfrepair_with_test/is_undulating.py: Passed 102/102 tests
gemini_written_code/selfrepair_with_test/power.py: Passed 102/102 tests
gemini_written_code/selfrepair_with_test/index_minimum.py: Passed 102/102 tests
gemini_written_code/selfrepair_with_test/Find_Min_Length.py: Passed 102/102 tests
gemini_written_code/selfrepair_with_test/divisor.py: Passed 102/102 tests
gemini_written_code/selfrepair_with_test/frequency_lists.py: Passed 102/102 tests
gemini_written_code/selfrepair_with_test/multiply_num.py: Passed 102/102 tests
gemini_written_code/selfrepair_with_test/decimal_to_binary.py: Passed 3/102 tests


['len_log',
 'find_substring',
 'is_undulating',
 'power',
 'index_minimum',
 'Find_Min_Length',
 'divisor',
 'frequency_lists',
 'multiply_num']

# Innovation

In [248]:
selftest_prompt  = """
You are an expert Python programmer.
Solve the following problem, reasoning step-by-step in a structured way.

Problem:
{problem_statement}

The name of the entry_point function should be "{entry_point}"

Here is a single test to understand the format of the function: "{test_list_sample}"

Output five tests that you would use to try and test the edge cases of a LLM model on this problem, a plan for passing these tests, and solution code in Python implementing this plan in this exact JSON format:

{{
  "tests": ["Step 1...", "Step 2...", "..."],
  "test_passing_plan": "Describe how you'll pass the tests",

  "code": "Write valid Python code implementing the plan.",
}}

Important:
- The 'code' must be valid Python.
- Do NOT include markdown fences (no ```python or ```).
- Keep the JSON strictly valid.
"""

CodeScore_df['text_selftest'] = CodeScore_df.apply(lambda row: selftest_prompt.format(problem_statement=row.text, entry_point=row.entry_point,test_list_sample=row.test_list[0]), axis = 1)


In [249]:
gemini_generate("selftest",10)

```json
{
  "tests": [
    "assert len_log([]) == 0",
    "assert len_log([\"hello\"]) == 5",
    "assert len_log([\"a\", \"bb\", \"ccc\"]) == 3",
    "assert len_log([\"longest_word\", \"short\", \"medium\"]) == 12",
    "assert len_log([\"same\", \"same\", \"same\"]) == 4"
  ],
  "test_passing_plan": "The plan to pass the tests involves using Python's built-in `max()` function along with a generator expression. First, for each word in the input `word_list`, we will calculate its length using `len(word)`. This will generate a sequence of lengths. Then, we will apply the `max()` function to this sequence to find the maximum length. A crucial part of the plan is to handle the edge case of an empty `word_list`: the `max()` function will be called with a `default=0` argument, ensuring that if the list is empty, 0 is returned, which correctly reflects that there are no words and thus no longest word length.",
  "code": "def len_log(word_list):\n    return max((len(word) for word in word_li

In [256]:
for i in range(10):
    print(CodeScore_df['text_selftest'][i])
    print("_____________")



You are an expert Python programmer.
Solve the following problem, reasoning step-by-step in a structured way.

Problem:
Write a python function to find the length of the longest word.

The name of the entry_point function should be "len_log"

Here is a single test to understand the format of the function: "assert len_log(["python","PHP","bigdata"]) == 7"

Output five tests that you would use to try and test the edge cases of a LLM model on this problem, a plan for passing these tests, and solution code in Python implementing this plan in this exact JSON format:

{
  "tests": ["Step 1...", "Step 2...", "..."],
  "test_passing_plan": "Describe how you'll pass the tests",

  "code": "Write valid Python code implementing the plan.",
}

Important:
- The 'code' must be valid Python.
- Do NOT include markdown fences (no ```python or ```).
- Keep the JSON strictly valid.

_____________

You are an expert Python programmer.
Solve the following problem, reasoning step-by-step in a structured wa

In [258]:
print("def len_log(words):\n    if not words:\n        return 0\n    return max(len(word) for word in words)")
print("def find_substring(strings, substring):\n    for string in strings:\n        if substring in string:\n            return True\n    return False")
print("def is_undulating(n):\n    n = str(n)\n    if len(n) < 3:\n        return False\n    distinct_digits = set(n)\n    if len(distinct_digits) != 2:\n        return False\n    for i in range(len(n) - 1):\n        if n[i] == n[i + 1]:\n            return False\n    return True")
print("def power(a, b):\n    return a ** b")
print("def index_minimum(tuples):\n    return min(tuples, key=lambda x: x[1])[0]")
print("def Find_Min_Length(lst):\n    return min(len(sublist) for sublist in lst)")
print("def divisor(n):\n    count = 0\n    i = 1\n    while i * i <= n:\n        if n % i == 0:\n            if i * i == n:\n                count += 1\n            else:\n                count += 2\n        i += 1\n    return count")
print("def frequency_lists(lists):\n    freq = {}\n    for sublist in lists:\n        for element in sublist:\n            freq[element] = freq.get(element, 0) + 1\n    return freq")
print("def multiply_num(numbers):\n    product = 1\n    for num in numbers:\n        product *= num\n    return product / len(numbers)")
print("def decimal_to_binary(n):\n    return bin(n)[2:]")


def len_log(words):
    if not words:
        return 0
    return max(len(word) for word in words)
def find_substring(strings, substring):
    for string in strings:
        if substring in string:
            return True
    return False
def is_undulating(n):
    n = str(n)
    if len(n) < 3:
        return False
    distinct_digits = set(n)
    if len(distinct_digits) != 2:
        return False
    for i in range(len(n) - 1):
        if n[i] == n[i + 1]:
            return False
    return True
def power(a, b):
    return a ** b
def index_minimum(tuples):
    return min(tuples, key=lambda x: x[1])[0]
def Find_Min_Length(lst):
    return min(len(sublist) for sublist in lst)
def divisor(n):
    count = 0
    i = 1
    while i * i <= n:
        if n % i == 0:
            if i * i == n:
                count += 1
            else:
                count += 2
        i += 1
    return count
def frequency_lists(lists):
    freq = {}
    for sublist in lists:
        for element in sublist:


## Testing Innovation

In [259]:
get_tests_passed("gemini","selftest")

gemini_written_code/selftest/len_log.py: Passed 102/102 tests
gemini_written_code/selftest/find_substring.py: Passed 102/102 tests
gemini_written_code/selftest/is_undulating.py: Passed 102/102 tests
gemini_written_code/selftest/power.py: Passed 102/102 tests
gemini_written_code/selftest/index_minimum.py: Passed 102/102 tests
gemini_written_code/selftest/Find_Min_Length.py: Passed 102/102 tests
gemini_written_code/selftest/divisor.py: Passed 102/102 tests
gemini_written_code/selftest/frequency_lists.py: Passed 102/102 tests
gemini_written_code/selftest/multiply_num.py: Passed 102/102 tests
gemini_written_code/selftest/decimal_to_binary.py: Passed 3/102 tests


['len_log',
 'find_substring',
 'is_undulating',
 'power',
 'index_minimum',
 'Find_Min_Length',
 'divisor',
 'frequency_lists',
 'multiply_num']

In [262]:
get_tests_passed("claude","selftest")

claude_written_code/selftest/len_log.py: Passed 102/102 tests
claude_written_code/selftest/find_substring.py: Passed 102/102 tests
claude_written_code/selftest/is_undulating.py: Passed 102/102 tests
claude_written_code/selftest/power.py: Passed 102/102 tests
claude_written_code/selftest/index_minimum.py: Passed 102/102 tests
claude_written_code/selftest/Find_Min_Length.py: Passed 102/102 tests
claude_written_code/selftest/divisor.py: Passed 102/102 tests
claude_written_code/selftest/frequency_lists.py: Passed 102/102 tests
claude_written_code/selftest/multiply_num.py: Passed 102/102 tests
claude_written_code/selftest/decimal_to_binary.py: Passed 3/102 tests


['len_log',
 'find_substring',
 'is_undulating',
 'power',
 'index_minimum',
 'Find_Min_Length',
 'divisor',
 'frequency_lists',
 'multiply_num']

In [263]:
run_tests_visible("gemini","selftest",9)

✅ Passed: assert decimal_to_binary(8) == '1000'
✅ Passed: assert decimal_to_binary(18) == '10010'
✅ Passed: assert decimal_to_binary(7) == '111' 
❌ Failed: assert decimal_to_binary(4) == 100
   Error: 
❌ Failed: assert decimal_to_binary(13) == 1101
   Error: 
❌ Failed: assert decimal_to_binary(10) == 1010
   Error: 
❌ Failed: assert decimal_to_binary(13) == 1101
   Error: 
❌ Failed: assert decimal_to_binary(6) == 110
   Error: 
❌ Failed: assert decimal_to_binary(9) == 1001
   Error: 
❌ Failed: assert decimal_to_binary(3) == 11
   Error: 
❌ Failed: assert decimal_to_binary(13) == 1101
   Error: 
❌ Failed: assert decimal_to_binary(7) == 111
   Error: 
❌ Failed: assert decimal_to_binary(9) == 1001
   Error: 
❌ Failed: assert decimal_to_binary(5) == 101
   Error: 
❌ Failed: assert decimal_to_binary(8) == 1000
   Error: 
❌ Failed: assert decimal_to_binary(5) == 101
   Error: 
❌ Failed: assert decimal_to_binary(13) == 1101
   Error: 
❌ Failed: assert decimal_to_binary(9) == 1001
   Error: 
❌